In [1]:
#作ったモデルを動かしてみる
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()


/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path="../../models/hf"
model_path="../../models/hf/0405_100m_clean_ja"
model_path="../../models/hf/0405_2700m_clean_ja"
model_path="../../models/llama/hf"
model_path="../../models/llama/conv_hf"
model = AutoModelForCausalLM.from_pretrained(model_path,device_map="auto")

Loading checkpoint shards: 100%|██████████| 2/2 [00:15<00:00,  7.94s/it]


In [3]:
for name, param in model.named_parameters():
    print(name)

model.embed_tokens.weight
model.layers.0.self_attn.q_proj.weight
model.layers.0.self_attn.k_proj.weight
model.layers.0.self_attn.v_proj.weight
model.layers.0.self_attn.o_proj.weight
model.layers.0.mlp.gate_proj.weight
model.layers.0.mlp.up_proj.weight
model.layers.0.mlp.down_proj.weight
model.layers.0.input_layernorm.weight
model.layers.0.post_attention_layernorm.weight
model.layers.1.self_attn.q_proj.weight
model.layers.1.self_attn.k_proj.weight
model.layers.1.self_attn.v_proj.weight
model.layers.1.self_attn.o_proj.weight
model.layers.1.mlp.gate_proj.weight
model.layers.1.mlp.up_proj.weight
model.layers.1.mlp.down_proj.weight
model.layers.1.input_layernorm.weight
model.layers.1.post_attention_layernorm.weight
model.layers.2.self_attn.q_proj.weight
model.layers.2.self_attn.k_proj.weight
model.layers.2.self_attn.v_proj.weight
model.layers.2.self_attn.o_proj.weight
model.layers.2.mlp.gate_proj.weight
model.layers.2.mlp.up_proj.weight
model.layers.2.mlp.down_proj.weight
model.layers.2.inp

In [4]:
model.modules

<bound method Module.modules of LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(65024, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
  

In [6]:

tokenizer = AutoTokenizer.from_pretrained(model_path)

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [7]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, max_new_tokens=200, repetition_penalty=1.5)

In [8]:
import time
text_list=["今日はいい",
"富士山は",
"質問: 今日の天気は? 回答:",
"批判的な",
"大規模言語モデルは",
"AI研究の問題点は",
"化学研究の問題点は",
"I have a",
"Functional polymers are",
"機能性高分子は",           
"ホンダ フィット 販売",
"英語: He is a good man. 日本語: ",
]


for text in text_list:
    perp=perplexity(model,tokenizer,text)
    s_time=time.time()
    res=pipe(text)[0]["generated_text"]
    consumed_time=time.time()-s_time
    print("-------")
    print("input: ", text)
    print("perplexity: ",perp)
    print("time: ", consumed_time)
    print("time/character: ", consumed_time/len(res))
    print("output: ",res)

-------
input:  今日はいい
perplexity:  254.3335723876953
time:  5.927626371383667
time/character:  0.0073726696161488395
output:  今日はいい、その後にこのような人がいると思った。
「......そう言って」と言うのを聞いてからも分かる通りだったのだろうけれど、「私には何もできないわ!」という言葉で返したように見えたようだ。「でもね......」と言いながら自分の中では考えられないほどのものだと思うし......。  Title: How to Get a Boyfriend on Club Penguins and Your iPhone or iPad

Introduction: This guide is designed for children, as well as any other items that may have been used in the game. By following these steps carefully, you can create an engaging yet informative appearance while maintaining your overall aesthetic appeal without causing unnecessary distress or discomfort due to unforeseen circumstances such as excessive force exposure (such as fireproof), water damaged areas like cotton balls or wool sticks with plastic wraprs; however, this tutorial aims at creating safe environments where they might be more
-------
input:  富士山は
perplexity:  40546.73046875
time:  5.177065134048462
time/character:  0.00721039712263016

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


-------
input:  ホンダ フィット 販売
perplexity:  13512.64453125
time:  5.18306040763855
time/character:  0.007457640874300071
output:  ホンダ フィット 販売, and the other is a common feature for each of these two-component systems. Therefore we can use this to find an efficient solution that will be used in order to get more precise results than using one or both (or all) inputs as well; however if you are not ablely sure what happens when there's no reason why it works out so far:
• If your system hasn’t been done yet then I would have made some mistake about its own behavior with my code! – user2564 Mar 17 '18 at 09:31 # -*- coding: utf-8 -*-
# Copyright(C) 2015 Google LLC <http://www.opensourceforgeer>
from __future__ import unicode_literals
import sys
sys.path = osp.join('~') + '/*.py', encoding='utf-8').read()[:-1]+'.txt'.
-------
input:  英語: He is a good man. 日本語: 
perplexity:  533.469482421875
time:  5.196166276931763
time/character:  0.006306027035111362
output:  英語: He is a good man. 日本語: 19-y

In [ ]:
tokenizer